# 🧭 LLM’e Giden Yol Haritası (Katman Katman)

| Aşama   | Katman / Kavram                 | Ne Öğreneceğiz                                      | Kodla mı?         |
| ------- | ------------------------------- | --------------------------------------------------- | ----------------- |
| **1️⃣** | **Tokenization**                | Metni sayılara dönüştürmek (subword, BPE)           | ✅                 |
| **2️⃣** | **Embedding Layer**             | Token ID → Vektör                                   | ✅                 |
| **3️⃣** | **Positional Encoding**         | Sıra bilgisini modele eklemek                       | ✅                 |
| **4️⃣** | **Self-Attention (Tek başına)** | Dikkat mekanizmasının özü (Q, K, V)                 | ✅                 |
| **5️⃣** | **Multi-Head Attention**        | Paralel dikkat başlarıyla temsil gücü               | ✅                 |
| **6️⃣** | **Feed-Forward Network (MLP)**  | Her token’ın bilgisine bağımsız dönüşüm             | ✅                 |
| **7️⃣** | **Residual + LayerNorm**        | Stabilite ve gradient akışı                         | ✅                 |
| **8️⃣** | **Transformer Block**           | Yukarıdakilerin birleşimi                           | ✅                 |
| **9️⃣** | **LLM Base Model (GPT-style)**  | Embedding + N adet Transformer Block + Output Head  | ✅                 |
| **🔟**  | **Training Loop & Masking**     | Causal mask, loss hesaplama, autoregressive öğrenme | ✅                 |
| **⚡**   | **Inference (Text Generation)** | Greedy, sampling, top-k/p decoding                  | ✅                 |
| **🚀**  | **Gelişmişler**                 | KV Cache, RoPE, FlashAttention, LoRA                | ⚙️ (isteğe bağlı) |


# 🧱 LLM Modelini Katman Katman İnşa Planı

| Aşama | Katman                         | Amaç                                         |
| ----- | ------------------------------ | -------------------------------------------- |
| 1️⃣   | **Embedding Layer**            | Token id’lerini sürekli vektör uzayına taşır |
| 2️⃣   | **Positional Encoding**        | Sıralama bilgisini modele katar              |
| 3️⃣   | **Self-Attention (QKV)**       | Token’ların birbirine dikkat etmesini sağlar |
| 4️⃣   | **Multi-Head Attention**       | Bilgiyi farklı alt uzaylarda işler           |
| 5️⃣   | **Feed Forward Network (MLP)** | Her token bilgisini ayrı dönüştürür          |
| 6️⃣   | **Residual + LayerNorm**       | Öğrenmeyi stabilize eder                     |
| 7️⃣   | **Transformer Block**          | Yukarıdakilerin birleşimi                    |
| 8️⃣   | **LLM (GPT-style)**            | Embedding + N blok + çıkış projeksiyonu      |


---

# 🔹 Başlangıç: 1️⃣ Embedding Katmanı
🎯 Amaç:

* Token ID → vektör uzayına dönüşüm.
* Her kelime için öğrenilebilir bir temsil vektörü.

In [2]:
import torch
import torch.nn as nn

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, embed_dim: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)

    def forward(self, x):
        # x: [batch_size, seq_len]
        return self.embedding(x)  # [batch_size, seq_len, embed_dim]


# 🔹 2️⃣ Positional Encoding (Sin/Cos)

* Transformers sırayı doğal olarak bilmez, bu yüzden pozisyon bilgisi eklememiz gerekir.

🧠 Teori

* Pozisyon bilgisi sinüs ve kosinüs fonksiyonlarıyla kodlanır:

In [3]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim: int, max_len: int = 512):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


# 3️⃣ Self-Attention (Tek Başlı)
🧠 Teori

* Her token, diğer token’lara dikkat (attention) uygular:

In [4]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super().__init__()
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.scale = embed_dim ** -0.5

    def forward(self, x, mask=None):
        Q = self.q_proj(x)
        K = self.k_proj(x)
        V = self.v_proj(x)
        scores = (Q @ K.transpose(-2, -1)) * self.scale
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = scores.softmax(dim=-1)
        return attn @ V


# 🔹 4️⃣ Multi-Head Attention

* Birden fazla attention başı paralel çalışır.
* Her biri farklı bir alt uzayda dikkat öğrenir.

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.head_dim = embed_dim // num_heads
        self.num_heads = num_heads
        self.qkv = nn.Linear(embed_dim, embed_dim * 3)
        self.out = nn.Linear(embed_dim, embed_dim)
        self.scale = self.head_dim ** -0.5

    def forward(self, x, mask=None):
        B, T, C = x.shape
        qkv = self.qkv(x).reshape(B, T, 3, self.num_heads, self.head_dim)
        q, k, v = qkv.unbind(2)  # her biri [B, T, num_heads, head_dim]
        scores = (q @ k.transpose(-2, -1)) * self.scale
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = scores.softmax(dim=-1)
        out = (attn @ v).transpose(1, 2).reshape(B, T, C)
        return self.out(out)


# 🧩 5️⃣ Feed Forward (MLP) Katmanı
🎯 Amaç

* Her token’ın temsilini bağımsız olarak dönüştürmek.
* Self-Attention, token’lar arası etkileşimi yakalarken;
* FeedForward, her token’ın kendi iç temsiline dönüşüm uygular.
> Bu yapı sayesinde model non-linearity kazanır ve her token kendi bilgisi üzerinde çalışabilir.

In [6]:
class FeedForward(nn.Module):
    def __init__(self, embed_dim: int, expansion: int = 4, dp: float = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embed_dim, expansion * embed_dim),
            nn.ReLU(),
            nn.Linear(expansion * embed_dim, embed_dim),
            nn.Dropout(dp)
        )

    def forward(self, x):
        return self.net(x)


# ⚖️ 6️⃣ Layer Normalization + Residual Connection
🎯 Amaç

* Derin ağlarda gradyan akışını stabilize etmek ve öğrenmeyi dengede tutmak.

* Residual Connection: Giriş + Çıkış → 

* LayerNorm: Katman bazlı normalizasyon (her örnekte tüm embedding boyutu için)

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, layer: nn.Module, embed_dim: int):
        super().__init__()
        self.layer = layer
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x, *args, **kwargs):
        return x + self.layer(self.norm(x, *args, **kwargs))

# 🧱 7️⃣ Transformer Block (Tam Hal)

Artık elimizde:

* MultiHeadAttention

* FeedForward

* Residual + LayerNorm

Bunları birleştirip tek bir Transformer bloğu yapıyoruz.

* Bu bloğu N kez üst üste koyduğumuzda → LLM oluşur.

In [8]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int, dp: float = 0.1):
        super().__init__()
        self.attn = MultiHeadAttention(embed_dim, num_heads)
        self.ff = FeedForward(embed_dim, dp=dp)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dp = nn.Dropout(dp)

    def forward(self, x, mask=None):
        # 1️⃣ Attention + Residual
        attn_out = self.attn(self.norm1(x), mask=mask)
        x = x + self.dp(attn_out)

        # 2️⃣ FeedForward + Residual
        ff_out = self.ff(self.norm2(x))
        x = x + self.dp(ff_out)
        return x

---
## Geçen REPO'da yaptıklarımıza bakalım.O modeli aşağıya bırakıyorum.Bu model üzerinden geliştirmelere bakayacağız.Eğer o repoyu incelemediyseniz aşağıya bağlantıyı bırakıyorum;

> ### "https://github.com/huseyin-dgn/PyTorch-Bolum_5---SeqtoSeq_Transformers"

----

### Bahsettiğim model kodunu aşağıya tekrar yazalım ; 

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Optional


class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=None, dp=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads or max(1, embed_dim // 64)
        self.head_dim = embed_dim // self.num_heads
        self.scale = math.sqrt(self.head_dim)

        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dp)

    def split_heads(self, x):
        B, S, E = x.size()
        return x.view(B, S, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, x):
        x = x.transpose(1, 2).contiguous()
        B, S, _, _ = x.shape
        return x.view(B, S, self.embed_dim)

    def forward(self, query, key, value, key_padding_mask=None, attn_mask=None):
        Q = self.split_heads(self.q_proj(query))
        K = self.split_heads(self.k_proj(key))
        V = self.split_heads(self.v_proj(value))

        scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scale

        if attn_mask is not None:
            if attn_mask.dim() == 2:
                scores += attn_mask.unsqueeze(0).unsqueeze(0)
            elif attn_mask.dim() == 3:
                scores += attn_mask.unsqueeze(1)

        if key_padding_mask is not None:
            mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
            scores = scores.masked_fill(mask, torch.finfo(scores.dtype).min)

        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        out = torch.matmul(attn, V)
        out = self.combine_heads(out)
        return self.out_proj(out)
    
class EnchancedLSTMEncoder(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int = 256, hidden_size: int = 512,
                 num_layers: int = 2, bidirectional: bool = True, dp: float = 0.1,
                 use_attn: bool = True, pre_norm: bool = True):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.directions = 2 if bidirectional else 1
        self.use_attn = use_attn
        self.pre_norm = pre_norm
        self.dropout = dp

        lstm_input_dim = hidden_size * self.directions
        self.residual_proj = nn.Linear(embedding_dim, lstm_input_dim)

        self.lstm = nn.LSTM(
            input_size=lstm_input_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dp if num_layers > 1 else 0.0,
            bidirectional=bidirectional,
            batch_first=True
        )

        self.layer_norm_in = nn.LayerNorm(lstm_input_dim)
        self.layer_norm_out = nn.LayerNorm(lstm_input_dim)
        self.attention = MultiHeadAttention(lstm_input_dim) if use_attn else None

        self.ffn = nn.Sequential(
            nn.Linear(lstm_input_dim, lstm_input_dim * 4),
            nn.GELU(),
            nn.Dropout(dp),
            nn.Linear(lstm_input_dim * 4, lstm_input_dim),
            nn.Dropout(dp)
        )

    def forward(self, src: torch.Tensor, src_key_padding_mask: Optional[torch.Tensor] = None,
                attn_mask: Optional[torch.Tensor] = None):

        embedded = self.embedding(src)
        residual = self.residual_proj(embedded)
        lstm_input = self.layer_norm_in(residual) if self.pre_norm else residual

        outputs, (hidden, cell) = self.lstm(lstm_input)
        outputs = self.layer_norm_out(outputs)

        if self.attention:
            attn_out = self.attention(outputs, outputs, outputs,
                                      key_padding_mask=src_key_padding_mask,
                                      attn_mask=attn_mask)
            outputs = outputs + attn_out

        outputs = outputs + self.ffn(outputs)
        return outputs, hidden, cell
    

class MultiHeadAttention(nn.Module):
    def __init__(self,embed_dim : int , num_heads:Optional[int] =None , dp:float = 0.3):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads or max(1, embed_dim//64)
        self.head_dim = embed_dim // self.num_heads
        assert embed_dim % self.num_heads == 0 , "embed_idm ve num_heads bölünemiyor"
        self.scale = math.sqrt(self.head_dim)

        self.q_proj = nn.Linear(embed_dim , embed_dim)
        self.v_proj = nn.Linear(embed_dim , embed_dim)
        self.k_proj = nn.Linear(embed_dim,embed_dim)
        self.out_proj = nn.Linear(embed_dim,embed_dim)
        self.dropout = nn.Dropout(dp)

    def split_heads(self,x):
        B,S,E = x.size()
        return x.view(B,S,self.num_heads , self.head_dim).transpose(1,2)

    def combine_heads(self,x):
        x = x.transpose(1,2).contiguous()
        B,S,_,_ = x.size()
        return x.view(B,S,self.embed_dim)
    
    def forward(self,query,key,value,key_padding_mask = None , attn_mask = None):
        Q = self.split_heads(self.q_proj(query))
        V = self.split_heads(self.v_proj(value))
        K = self.split_heads(self.k_proj(key))

        scores = torch.matmul(Q,K.transpose(-2,-1)) / self.scale

        if attn_mask is not None:
            if attn_mask.dim() == 2:
                scores = scores + attn_mask.unsqueeze(0).unsqueeze(0)
            elif attn_mask.dim() == 3:
                scores = scores + attn_mask.unsqueeze(1)

        
        if key_padding_mask is not None:
            mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
            scores = scores.masked_fill(mask, torch.finfo(scores.dtype).min)


        attn = F.softmax(scores , dim = -1 )
        attn = self.dropout(attn)

        out = torch.matmul(attn , V)
        out = self.combine_heads(out)

        return self.out_proj(out)
    
def init_decoder_states_from_encoder(hidden, cell, num_decoder_layers, decoder_hidden_size,
                                     encoder_directions=1, proj_layer=None):
    num_enc_total, B, enc_hidden = hidden.size()
    num_enc_layers = num_enc_total // encoder_directions

    h = hidden.view(num_enc_layers, encoder_directions, B, enc_hidden)
    c = cell.view(num_enc_layers, encoder_directions, B, enc_hidden)

    # Bidirectional merge
    h_merged = h.mean(dim=1)  # (num_layers, B, enc_hidden)
    c_merged = c.mean(dim=1)

    if proj_layer is None and enc_hidden != decoder_hidden_size:
        proj_layer = nn.Linear(enc_hidden, decoder_hidden_size).to(hidden.device)

    if proj_layer is not None:
        L = num_enc_layers
        # ❌ Düzeltme: directions ile çarpma yok
        h_proj = proj_layer(h_merged.view(L * B, enc_hidden)).view(L, B, decoder_hidden_size)
        c_proj = proj_layer(c_merged.view(L * B, enc_hidden)).view(L, B, decoder_hidden_size)
    else:
        h_proj, c_proj = h_merged, c_merged

    if num_decoder_layers == num_enc_layers:
        return h_proj, c_proj
    elif num_decoder_layers > num_enc_layers:
        h_final = torch.cat([h_proj[min(i, num_enc_layers - 1):min(i, num_enc_layers - 1) + 1]
                             for i in range(num_decoder_layers)], dim=0)
        c_final = torch.cat([c_proj[min(i, num_enc_layers - 1):min(i, num_enc_layers - 1) + 1]
                             for i in range(num_decoder_layers)], dim=0)
        return h_final, c_final
    
class GatedFFN(nn.Module):
    def __init__(self, hidden_size:int , dp:float = 0.2):
        super().__init__()
        self.fc1= nn.Linear(hidden_size , hidden_size * 4)
        self.fc2 = nn.Linear(hidden_size * 4 ,hidden_size)
        self.gate = nn.Linear(hidden_size , hidden_size)
        self.act = nn.GELU()
        self.dropout = nn.Dropout(dp)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        out = self.fc2(self.act(self.fc1(x)))
        gate = self.sigmoid(self.gate(x))
        return self.dropout(out + gate)
    
class PositionelEncoding(nn.Module):
    def __init__(self, hidden_size : int , max_len : int = 5000):
        super().__init__()
        pe = torch.zeros(max_len , hidden_size)
        position = torch.arange(0 , max_len , dtype = torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0 , hidden_size,2).float() *  (-math.log(100000.0) / hidden_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe' , pe)
    
    def forward(self,x):
        seq_len = x.size(1)
        return x + self.pe[: , :seq_len , :]
    
class AdaptiveDecoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim = 256,hidden_size =512, num_layers=3,dp=0.3 , use_attn =True , pre_norm = True , max_len = 5000):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size , embedding_dim)
        self.hidden_size = hidden_size
        self.pre_norm = pre_norm
        self.use_attn = use_attn
        self.dropout = nn.Dropout(dp)
        self.num_layers = num_layers

        self.input_proj = nn.Linear(embedding_dim,hidden_size)
        self.pos_encoding = PositionelEncoding(hidden_size , max_len=max_len)

        self.lstm = nn.LSTM(hidden_size , hidden_size , num_layers=num_layers , dropout=dp  if num_layers>1 else 0.0 , batch_first=True)

        self.layer_norm_residual = nn.LayerNorm(hidden_size)
        self.attention = MultiHeadAttention(hidden_size) if use_attn else None
        self.gated_ffn = GatedFFN(hidden_size , dp)
        self.layer_norm_ffn = nn.LayerNorm(hidden_size)

    def forward(self, x, hidden ,cell , src_key_padding = None , encoder_output = None , attn_mask = None):
        embedded = self.input_proj(self.embedding(x))
        embedded = self.pos_encoding(embedded)
        lstm_input = self.layer_norm_residual(embedded) if self.pre_norm else embedded

        outputs , (hidden , cell) = self.lstm(lstm_input , (hidden , cell))

        if self.attention and encoder_output is not None:
            attn_out = self.attention(outputs , encoder_output , encoder_output,key_padding_mask = src_key_padding , attn_mask = attn_mask)
            outputs = outputs + attn_out
        
        outputs = outputs + self.gated_ffn(outputs)
        outputs = self.layer_norm_ffn(outputs)

        return outputs , hidden , cell

---

## Şimdi bu modeli LLM'e entegre etmeye çalışalım  ;

---

# 🔹 SEQ2SEQ Modeli İncelemesi ve LLM Uyarlama Analizi

Bu hücre, gönderilen SEQ2SEQ kodunu temel alarak **LLM tarzı bir Transformer’a dönüşüm adımlarını** özetler.



## 1️⃣ Encoder (EnchancedLSTMEncoder)

- **Embedding Layer:** `self.embedding` → token ID’lerini vektöre çevirir.
- **Residual Projection:** `self.residual_proj` → embedding boyutunu LSTM giriş boyutuna eşler.
- **LayerNorm:** `self.layer_norm_in` / `self.layer_norm_out` → pre/post norm işlemleri.
- **LSTM:** Çok katmanlı, bidirectional LSTM.
- **Attention:** Opsiyonel `MultiHeadAttention`.
- **FeedForward:** `self.ffn` → Gated / GELU aktivasyonlu MLP.

**💡 LLM uyarlaması:**
- LSTM yerine **Transformer Encoder Block** kullanılacak.
- Residual + LayerNorm + Attention + FFN yapısı zaten hazır; doğrudan Transformer blok mantığına uyarlanabilir.



## 2️⃣ Decoder (AdaptiveDecoder)

- **Embedding + Input Projection:** `self.embedding` + `self.input_proj`.
- **Positional Encoding:** `self.pos_encoding`.
- **LSTM:** Çok katmanlı decoder LSTM.
- **Attention:** Encoder-Decoder attention (`MultiHeadAttention`).
- **Gated FFN:** `self.gated_ffn`.
- **LayerNorm:** `self.layer_norm_ffn`, `self.layer_norm_residual`.

**💡 LLM uyarlaması:**
- Decoder zaten **Transformer decoder mantığına yakın**:
  - LSTM → Masked Self-Attention + FeedForward
  - Encoder-Decoder Attention → Cross-Attention
  - GatedFFN → klasik FFN yerine kullanılabilir
- Positional Encoding ve Residual + LayerNorm yapısı korunabilir.



## 3️⃣ MultiHeadAttention

- Mevcut hem encoder hem decoder için ortak `MultiHeadAttention`.
- Split/Combine heads, QKV projeksiyonu, mask handling mevcut.
- Ufak değişikliklerle **Transformer standard attention** (pre-norm + causal mask) yapılabilir.



## 4️⃣ LLM Mimarisine Geçiş Notları

| Mevcut | LLM Uyarlaması |
|---------|----------------|
| LSTM tabanlı encoder | Transformer encoder block |
| LSTM tabanlı decoder | Transformer decoder block (masked self-attention) |
| Gated FFN / standard FFN | Transformer feedforward (ReLU/GELU) |
| Encoder-decoder attention | Cross-attention |
| Positional Encoding | Direkt kullanılabilir |
| LayerNorm + residual | Pre-norm yapısı olarak aynen kullanılabilir |



## ✅ Özet

- Kod zaten **modüler ve katmanlı**, LLM için gereken çoğu yapı mevcut.
- Yapılması gerekenler:
  1. **LSTM’leri kaldırıp Transformer blokları ile değiştirmek**
  2. **Decoder’da masked self-attention eklemek**
  3. **Cross-attention** mantığını korumak
  4. Mevcut **GatedFFN** veya FFN’i Transformer feedforward ile uyarlamak
- Bu sayede SEQ2SEQ modeli **GPT-style LLM** mantığıyla çalışabilir.


---